In [1]:
import numpy as np 
import pandas as pd 
from scipy.sparse import csr_matrix
import random
import math
import tensorflow as tf
from copy import deepcopy
import time
import wandb

In [2]:
wandb.login(key = "", relogin = True) # enter wandb key here
wandb.init(project="Agent_based_health_care_simulation", reinit=True)
wandb.run.name = "Simulation-" + str(time.time())

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: martinwessel. Use `wandb login --relogin` to force relogin


In [ ]:
class HealthCareSimulation:
    '''
    Overall score:
    Patient Sharing: 30%
    Distance: 20%
    Contact Availability: 10%
    Opening Hours: 10%
    Reputation: 10%
    Random: 30%
    '''
    def __init__(self, s=10, capacity= 0.2):
        
        data = np.load('/kaggle/input/demo-data/adj_all_doctors.npz')        # data contains x = [1,2,3,4,5]
        lst = {}
        for key in data.keys():
            lst[key] = data[key]
            
        matrix = csr_matrix((lst['data'], lst['indices'], lst['indptr']))
        self.shared_patients = matrix.toarray()
        self.diagonal = deepcopy(self.shared_patients.diagonal())
        self.df_capacity = pd.read_csv('/kaggle/input/demo-data/matched_and_imputed_doctors_with_capacity_threshold0.9.csv')
        # Distance 
        self.distance = np.random.randint(20, size=(200, 200))
        self.s = s # Lost patient Threshold
        self.capacity = capacity
    def setup(self):
        doctors = []
        for index, row in self.df_capacity.iterrows():
            r = random.randint(0,10) * 0.01
            doctors.append({'plz': row[0], 'hours': row[4], 'capacity': self.diagonal[index] + math.floor(self.diagonal[index]*self.capacity), 'rating': random.randint(0,4), 'information': random.randint(0,4), 'patients': [], 'status': True})

        total_citizens = self.diagonal.sum()
        print(f'Total amount of citizens: {total_citizens}')
        patient = 0
        for index, d in enumerate(doctors):
            d['patients'].extend([ele for ele in range(patient, patient + self.diagonal[index])])
            patient += self.diagonal[index]
            if len(d['patients']) > d['capacity']:
                print(f'Overflow Doctor {index}')
        return doctors, total_citizens
              
    def moving_prob(self, doctor, doctors, lst):
        # Distance
        d_dist = self.distance[doctor]
        prob_dist = np.array([20]*200) - ((d_dist/30)*20)
        # Rating and Information and opening hours
        opening = []
        rat_inf_prob = []
        for doc in doctors:
            ri_prob = doc['rating'] * 2.5 + doc['information'] * 2.5
            rat_inf_prob.append(ri_prob)
            opening.append(doc['hours'])
        opening_prob = [(ele / max(opening))* 10 for ele in opening]
        prob_dist += rat_inf_prob
        prob_dist += opening_prob
        # Patient sharing network
        shared = deepcopy(self.shared_patients[doctor])
        shared[doctor] = 0
        shared_dist = (shared / max(shared)) * 30
        prob_dist += shared_dist
        # Random portion
        rand_dist = np.random.randint(30, size=(200))
        prob_dist += rand_dist
        prob_dist[doctor] = 0 # Don't want the patient to go back to the original doctor
        for i, d in enumerate(doctors):
            if d['status'] == False:
                prob_dist[i] = 0
        for i in lst:
            prob_dist[i] = 0
        probabilities = tf.nn.softmax(prob_dist).numpy()
        return probabilities
    
    def measure_capacity(self, doctors):
        doc = pd.DataFrame(doctors).sum()
        cap = doc[2]
        pat = len(doc[5])
        print(f'Used capacity currently: {pat/cap}')
        return pat/cap

              
    def main(self, steps):
        doctors, total_citizens = self.setup()
        # Step in the simulation:
        lost_patients = 0
        #Take out doctor:
        curr_searching = []

        for step in range(steps):
            capa = self.measure_capacity(doctors)
            wandb.log({"Occupied_capacity": capa, "Step": step})
            while True:
                to_doctor = random.randint(0,199)
                print(f'STEP {step} Doctor {to_doctor} taken out.')
                if doctors[to_doctor]['status'] == True:
                    doctors[to_doctor]['status'] == False
                    break
            wo_patients = doctors[to_doctor]['patients'] 
            for i in wo_patients:
                curr_searching.append((i, to_doctor, []))
            print(f'{len(curr_searching)} patients are looking for a doctor.')
            wandb.log({"Searching_after_doctor_taken_out": len(curr_searching), "Step": step})
            # Searching Mechanism
            successful = []
            for index, patient in enumerate(curr_searching):
                weights = self.moving_prob(patient[1], doctors, patient[2])
                doctor_try = random.choices(range(200), weights=weights, k=1)[0]
                # Check if doctor has capacity
                if len(doctors[doctor_try]['patients']) < doctors[doctor_try]['capacity']:
                    doctors[doctor_try]['patients'].append(patient[0])
                    successful.append(index)
                    #print(patient, doctor_try)
                else:
                    patient[2].append(doctor_try)
            for i in sorted(successful, reverse=True):
                curr_searching.pop(i)
            print(f"{len(curr_searching)} are still searching after this step")
            wandb.log({"Still_searching": len(curr_searching), "Step": step})
            # Search for Lost Patients:
            l_i = []
            for i, j in enumerate(curr_searching):
                if len(j[2]) >= self.s:
                    lost_patients += 1
                    l_i.append(i)
            for i in sorted(l_i, reverse=True):
                curr_searching.pop(i)
            lp_share = lost_patients / total_citizens
            print(f"Currently Lost Patients: {lost_patients}. {lp_share} of all patients.")
            wandb.log({"Lost_Patients": lost_patients, "Step": step})
            wandb.log({"Lost_Patients_Share": lp_share, "Step": step})
    

In [ ]:
simulation = HealthCareSimulation()
simulation.main(200)
wandb.finish()